In [1]:
import fasttext
import fasttext.util
import gensim
from gensim.models import FastText

C:\Users\josem\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#Grado similaridad
#Numero de sinonimos en común
#Número de antónimos en común
#Relación con los sinonimos
#Relación con los antónimos
#Palabras comunes en la definición
#Familia semántica
#Número de palabras entre la pista y la evaluada(s)

In [6]:
%%capture
FAST_TEXT_MODEL = "cc.en.300.bin" # Model name in fasttext

fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model(FAST_TEXT_MODEL)

In [7]:
ft.get_nearest_neighbors('life')

[(0.6984261274337769, 'life-'),
 (0.6904193162918091, 'lives'),
 (0.6745699644088745, 'life.But'),
 (0.6679530143737793, 'life.It'),
 (0.6632518768310547, 'life.The'),
 (0.6621710062026978, 'life.In'),
 (0.6610385775566101, 'life.Now'),
 (0.652584969997406, 'life.So'),
 (0.6445083022117615, 'life.That'),
 (0.643845796585083, 'life--and')]

In [8]:
model = FastText.load_fasttext_format(FAST_TEXT_MODEL)
model.wv.most_similar(positive=['red', 'green'])

<ipython-input-8-24f7e452dabd>:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format(FAST_TEXT_MODEL)


[('blue', 0.8273436427116394),
 ('yellow', 0.8116556406021118),
 ('purple', 0.7858150005340576),
 ('orange', 0.7306221723556519),
 ('greeen', 0.698380172252655),
 ('pink', 0.6954873204231262),
 ('green-colored', 0.6573896408081055),
 ('white', 0.6534684300422668),
 ('blue-green', 0.6492818593978882),
 ('green-ish', 0.6438031196594238)]